In [2]:
import numpy as np
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from keras.models import model_from_json

In [3]:
from scipy.io import loadmat

sub1_train_data = loadmat('train_data_sub1.mat')
sub1_train_label = loadmat('train_label_sub1.mat')
sub1_test_data = loadmat('test_data_sub1.mat')

# sub2_train_data = loadmat('train_data_sub2.mat')
# sub2_train_label = loadmat('train_label_sub2.mat')

# sub3_train_data = loadmat('train_data_sub3.mat')
# sub3_train_label = loadmat('train_label_sub3.mat')

sub1_train_data_arr = sub1_train_data["train_data"]
sub1_train_label_arr = sub1_train_label["train_dg"]
sub1_test_data_arr = sub1_test_data["test_data"]

# sub2_train_data_arr = sub2_train_data["train_data"]
# sub2_train_label_arr = sub2_train_label["train_dg"]

# sub3_train_data_arr = sub3_train_data["train_data"]
# sub3_train_label_arr = sub3_train_label["train_dg"]

In [4]:
sub1_train_data_arr = np.delete(sub1_train_data_arr, 54, 1)
sub1_test_data_arr = np.delete(sub1_test_data_arr, 54, 1)

In [5]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(sub1_train_data_arr)
X_train_scaled = scaler.transform(sub1_train_data_arr)
X_test_scaled = scaler.transform(sub1_test_data_arr)

In [6]:
model = Sequential()
model.add(Dense(512, input_dim=61, activation='sigmoid'))
model.add(Dense(256, activation='sigmoid'))
model.add(Dense(128, activation='sigmoid'))
#Output layer
model.add(Dense(5, activation='linear'))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
model.summary()

history = model.fit(X_train_scaled, sub1_train_label_arr, validation_split=0.1, epochs=20)

loss = history.history['loss']
print(loss)
np.savetxt('loss_testSub1_drop54.ascii',(loss))

2022-06-20 01:08:11.220419: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-20 01:08:11.220953: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               31744     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 645       
Total params: 196,613
Trainable params: 196,613
Non-trainable params: 0
_________________________________________________________________
Train on 360000 samples, validate on 40000 samples
Epoch 1/20
360000/360000 [==============================] - 191s 530us/step - loss: 0.9089 - mae: 0.5553 - val_loss: 1.0116 - val_mae: 0.5944
Epoch 2/20
360000/360000 [===========================

In [7]:
predictions = model.predict(X_test_scaled)
print("Predicted values are: ", predictions)
# print("Real values are: ", y_test)
# np.savetxt ('output1_test.ascii',(y_test))
np.savetxt ('outputSub1_drop54_test.ascii',(predictions))
# mse_neural, mae_neural = model.evaluate(X_test_scaled, y_test)
# print('Mean squared error from neural net: ', mse_neural)
# print('Mean absolute error from neural net: ', mae_neural)

Predicted values are:  [[-0.39140022 -0.29349697 -0.5068352  -0.47745273 -0.39100242]
 [-0.3608983  -0.2849123  -0.5284451  -0.4546164  -0.38549864]
 [-0.34838238 -0.2505514  -0.52181506 -0.38493112 -0.3972881 ]
 ...
 [-0.16143188 -0.69238067 -0.48360184 -0.5773777  -0.60718054]
 [-0.14816166 -0.7387056  -0.5120534  -0.5819162  -0.6591191 ]
 [-0.14595036 -0.7719569  -0.5233601  -0.5919265  -0.691022  ]]


In [8]:
model_json = model.to_json()
with open("modelSub1_drop54_test.json", "w") as json_file:
    json_file.write(model_json)
        # serialize weights to HDF5
    model.save_weights("modelSub1_drop54_test.h5")
    print("Saved model to disk")

Saved model to disk
